In [1]:
from nbodykit.lab import cosmology
import matplotlib.pyplot as plt
import numpy as np
from mcfit import P2xi
from scipy.integrate import quad 
from scipy.integrate import simps 
from scipy import special

##parameter set
sigma_8=0.834
bias=2.1
beta=0.5196231/bias


c = cosmology.Planck15
c=c.clone(P_k_max=10)
c=c.clone(Omega0_b=0.049)
c=c.clone(Omega0_cdm=(0.3175-0.049))
c=c.clone(h=0.6711)
c=c.clone(n_s=0.9624)
c=c.match(sigma8=sigma_8)




Plin = cosmology.LinearPower(c, redshift=0.)
Pnl = cosmology.HalofitPower(c, redshift=0)
PZel = cosmology.ZeldovichPower(c, redshift=0)
cf_nl = cosmology.CorrelationFunction(Pnl)
cf_zel = cosmology.CorrelationFunction(PZel)
cf_lin=cosmology.CorrelationFunction(Plin)

/home/lzhuoyang/anaconda3/envs/nbodykit-env/lib/python3.8/site-packages/mcfit/mcfit.py:126: UserWarning: The default value of lowring has been changed to False, set it to True if you cannot reproduce previous results
  warnings.warn("The default value of lowring has been changed to False, "


In [2]:
##set real-space 2PCF

rl = np.logspace(-1, np.log10(200), 100000)
y=cf_zel(rl)

from scipy.interpolate import interp1d
cfzel1 = interp1d(rl, y, kind="cubic")



/home/lzhuoyang/anaconda3/envs/nbodykit-env/lib/python3.8/site-packages/mcfit/mcfit.py:217: UserWarning: The default value of extrap has been changed to False, set it to True if you cannot reproduce previous results
  warnings.warn("The default value of extrap has been changed to False, "


In [3]:
def twopcf1d(r):
    return cfzel1(r)


def r2_multiple_twopcf1d(r):
    return (r**2)*twopcf1d(r)

def r4_multiple_twopcf1d(r):
    return (r**4)*twopcf1d(r)


def barred_1(r):
    simp_x=np.linspace(0.1,r,20)
    simp_y=r2_multiple_twopcf1d(simp_x)
    return (3/(r**3))*simps(simp_y,simp_x)
    #return (3/(r**3))*quad(r2_multiple_twopcf1d,0.01,r)[0]

def barred_2(r):
    simp_x=np.linspace(0.1,r,20)
    simp_y=r4_multiple_twopcf1d(simp_x)
    return (5/(r**5))*simps(simp_y,simp_x)
    #return (5/(r**5))*quad(r4_multiple_twopcf1d,0.01,r)[0]


In [4]:
##parameter f,b
parameter_fiducial=[0.5,2.32]

In [5]:

def fofz(z):
    return 0.519623








##2D-2PCF multipoles ell=0,2,4

def multipole_0(r,parameter):
    bias=parameter[1]
    f=parameter[0]
    
    #return (1+(2/3)*0.519623*sigma_8*1+(1/5)*((0.519623*sigma_8)**2))*twopcf1d(r)/(sigma_8**2)
    return ((sigma_8*bias)**2+(2/3)*f*bias*sigma_8**2+(1/5)*((f*sigma_8)**2))*twopcf1d(r)/(sigma_8**2)
    ##return twopcf1d(r)


def multipole_2(r,parameter):
    bias=parameter[1]
    f=parameter[0]
    return ((4/3)*f*bias*sigma_8**2+(4/7)*(f*sigma_8)**2)*(twopcf1d(r)/(sigma_8**2)-barred_1(r)/(sigma_8**2))
    ##return (twopcf1d(r)-barred_1(r))

def multipole_4(r,parameter):
    bias=parameter[1]
    f=parameter[0]
    return (8/35)*((f*sigma_8)**2)*(twopcf1d(r)/(sigma_8**2)+2.5*barred_1(r)/(sigma_8**2)-3.5*barred_2(r)/(sigma_8**2))
    ##return (twopcf1d(r)+2.5*barred_1(r)-3.5*barred_2(r))

In [6]:
from numpy.polynomial import polynomial, legendre

import sympy as sym
def LegendrePolynomials(N,x):

    if N == 0:
        return 1
    if N == 1:
        return x
    p0 = LegendrePolynomials(0,x)
    p1 = LegendrePolynomials(1,x)
    assert N>=2
    for i in range(1,N):
        p = (2*i+1)/(i+1)*x*p1 - i/(i+1)*p0
        p0 = p1
        p1 = p
    return sym.simplify(p1)


##2D-2PCF
def thrsd2pcf(r,mnu,parameter):
    multipole0=multipole_0(r,parameter)
    multipole2=multipole_2(r,parameter)
    multipole4=multipole_4(r,parameter)
    return float(multipole0*LegendrePolynomials(0,mnu)+multipole2*LegendrePolynomials(2,mnu)+multipole4*LegendrePolynomials(4,mnu))

In [7]:
## cylinder correlation function
def pcf_cyls(rho,mnu,parameter):
    H=rho*mnu
    R=rho*np.sqrt(1-mnu**2)


    def innerfunc(z):
        return thrsd2pcf(np.sqrt(R**2+z**2),z/(np.sqrt(R**2+z**2)),parameter)

    nsimps=12
    simpsx=np.linspace(-H,H,nsimps)
    simpsy=np.zeros(nsimps)
    for i in range(nsimps):
        simpsy[i]=innerfunc(simpsx[i])
    return simps(simpsy,simpsx)/(2*H)


## disk correlation function
def pcf_disk(rho,mnu,parameter):
    H=rho*mnu
    R=rho*np.sqrt(1-mnu**2)
    def innerfunc(r):
        return thrsd2pcf(np.sqrt(H**2+r**2),H/(np.sqrt(H**2+r**2)),parameter)*r

    nsimps=12
    simpsx=np.linspace(0,R,nsimps)
    simpsy=np.zeros(nsimps)
    for i in range(nsimps):
        simpsy[i]=innerfunc(simpsx[i])
    return 2*simps(simpsy,simpsx)/(R**2)